In [36]:
import random as r
import copy as c
import csv

#this is a list of english words I found online
with open('/home/azito/Downloads/wordsEn.txt') as words_file:
    words = set(word.strip().lower() for word in words_file)

class Monkey():
    avgwords = 0
    
    def __init__(self,  wchars = [('a',5),('b',5),('c',5),('d',5),('e',5),('f',5),('g',5),('h',5),('i',5),('j',5),('k',5),('l',5),('m',5),('n',5),('o',5),('p',5),('q',5),('r',5),('s',5),('t',5),('u',5),('v',5),('w',5),('x',5),('y',5),('z',5)]):
        self.wchars = wchars

#copy-pasted from http://stackoverflow.com/questions/3679694/a-weighted-version-of-random-choice
def weighted_choice(choices):
    total= sum(w for c, w in choices)
    x = r.uniform(0, total)
    upto = 0
    for c, w in choices:
        if upto + w >= x:
            return c
        upto += w
    
def lavg(list): #lavg = list average
    tot = sum(list)
    return tot/len(list)

def chk(word):
    return word.lower() in words

#returns a random string based on the weighted letter choices
#provided in 'wchars'
def wrand_string(n,wchars):
    string = " "
    for i in range(n):
        x = r.randrange(100)
        #20% chance of a space, but spaces cannot be consecutive
        if x < 20 and string[-1] != ' ': 
            string += " "
        #80% chance of a letter
        else:
            string += weighted_choice(wchars)
    return string

#produces random (weighted) strings and then checks them for words
#of length 3 letters to 7 letters.
def monkeys(i,wchars):
    total = wrand_string(i,wchars)
    words = []
    for q in range(2,7):
        for a in range(len(total)-q):
            s = total[a:a+q]
            if chk(s) == True and len(s) > 2:
                words.append(s)
    return words

#takes a monkey function and runs it a bunch of times ('smooth' times)
#with numchars of characters and the weighter or nonweighted letters in chars
#and returns the average number of worlden words found
def average(numchars,chars,smooth,wordlen):
    results = []
    for i in range(smooth):
        words = monkeys(numchars,chars)
        foo = len([x for x in words if len(x) == wordlen])
        results.append(foo)
    return lavg(results)

def evolve(members, generations):
    mems = {}
    for i in range(members): #seed the first generation of monkeys
        mems[i] = Monkey()
    for n in range(generations):
        for i in range(members):
            for x in range(len(mems[i].wchars)): #mutate the letter frequencies
                a = r.randrange(0,10)
                if a == 0:
                    mems[i].wchars[x] = (mems[i].wchars[x][0],abs(mems[i].wchars[x][1] + r.uniform(-0.1,.1)))	
            #how many words will the monkey make with those frequencies
            mems[i].avgwords = average(100,mems[i].wchars,10,3) + average(100,mems[i].wchars,10,4)
        #choose the monkey who made the most words
        parentgen = c.deepcopy(mems)
        for i in range(members):
            parent = se([(m,m.avgwords) for m in parentgen.values()])
            mems[i].avgwords = 0 #reset the monkeys
            mems[i].wchars = parent.wchars #the next generation all starts with the best monkey's frequencies
    return (max(mems.values(),key=lambda x:x.avgwords)).wchars

In [14]:
with open('test.csv', 'w') as csvfile:
    mywriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for i in range(0,1000,5):
        mywriter.writerow([i,average(i,reg_wchars,10,3),average(i,mychars,10,3)])

In [ ]:
evolve(50,1000)